# 🌐 LangGraph Subgraph 완벽 가이드

## 📚 개요

이 튜토리얼은 LangGraph의 **Subgraph(서브그래프)** 개념을 깊이 있게 다룹니다. Subgraph는 그래프 안에 또 다른 그래프를 포함시키는 강력한 기능으로, **멀티 에이전트 시스템**이나 **복잡한 워크플로우**를 구축할 때 필수적입니다.

## 🎯 학습 목표

1. **Subgraph 개념** 이해 - 그래프 안의 그래프
2. **Shared State Schemas** - 부모와 자식 그래프가 상태를 공유하는 방법
3. **Different State Schemas** - 독립적인 상태를 가진 서브그래프
4. **Persistence** - 서브그래프의 메모리 관리
5. **스트리밍과 모니터링** - 서브그래프 출력 추적
6. **실전 활용** - 멀티 에이전트 시스템 구축

## 🔑 핵심 개념 미리보기

```
┌─────────────────────────────────────────────────────────┐
│                  Parent Graph (부모 그래프)              │
│  ┌───────────────────────────────────────────────────┐  │
│  │            Subgraph (서브그래프)                  │  │
│  │  ┌─────┐    ┌─────┐    ┌─────┐                 │  │
│  │  │Node1│ -> │Node2│ -> │Node3│                 │  │
│  │  └─────┘    └─────┘    └─────┘                 │  │
│  └───────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────┘
```

## 💡 중요 원칙

> **"서브그래프는 재사용 가능한 워크플로우 컴포넌트다"**
> 
> _Subgraphs are reusable workflow components_

## 📋 목차

1. [환경 설정](#환경-설정)
2. [Part 1: Subgraph 기본 개념](#part-1-subgraph-기본-개념-)
3. [Part 2: Shared State Schemas](#part-2-shared-state-schemas-)
4. [Part 3: Different State Schemas](#part-3-different-state-schemas-)
5. [Part 4: Persistence와 메모리 관리](#part-4-persistence와-메모리-관리-)
6. [Part 5: Subgraph State 모니터링](#part-5-subgraph-state-모니터링-)
7. [Part 6: Stream Outputs](#part-6-stream-outputs-)
8. [Part 7: 실전 - 멀티 에이전트 시스템](#part-7-실전---멀티-에이전트-시스템-)
9. [마무리](#-축하합니다)

## 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "your-api-key")

print("✅ 환경 설정 완료!")

---

# Part 1: Subgraph 기본 개념 🎯

## 1.1 Subgraph란 무엇인가?

**Subgraph**는 다른 그래프 안에 포함될 수 있는 **독립적인 그래프**입니다. 이를 통해:

1. **모듈화**: 복잡한 워크플로우를 작은 단위로 분할
2. **재사용성**: 동일한 서브그래프를 여러 곳에서 재사용
3. **캡슐화**: 내부 로직을 숨기고 인터페이스만 노출
4. **멀티 에이전트**: 각 에이전트를 독립적인 서브그래프로 구현

### 통신 방식

부모 그래프와 서브그래프 간 통신 방식은 두 가지입니다:

- **Shared State Schemas**: 부모와 자식이 **같은 상태 키를 공유**
- **Different State Schemas**: **독립적인 상태**를 가지고 변환을 통해 통신

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph.state import StateGraph, START, END

# 간단한 서브그래프 예제
class SimpleState(TypedDict):
    """Simple state for demonstration"""
    message: str

# 서브그래프 정의
def create_simple_subgraph():
    """Create a simple subgraph"""
    # 서브그래프 빌더 생성
    subgraph_builder = StateGraph(SimpleState)
    
    # 서브그래프의 노드 정의
    def subgraph_node(state: SimpleState):
        # 서브그래프 내부에서 처리
        return {"message": state["message"] + " [서브그래프에서 처리됨]"}
    
    # 노드와 엣지 추가
    subgraph_builder.add_node("process", subgraph_node)
    subgraph_builder.add_edge(START, "process")
    subgraph_builder.add_edge("process", END)
    
    # 서브그래프 컴파일
    return subgraph_builder.compile()

# 부모 그래프 정의
parent_builder = StateGraph(SimpleState)

# 부모 그래프의 노드
def parent_node(state: SimpleState):
    # 부모 그래프에서 전처리
    return {"message": "부모: " + state["message"]}

# 서브그래프를 노드로 추가
subgraph = create_simple_subgraph()
parent_builder.add_node("parent_node", parent_node)
parent_builder.add_node("subgraph_node", subgraph)  # 서브그래프를 노드로 추가!

# 엣지 연결
parent_builder.add_edge(START, "parent_node")
parent_builder.add_edge("parent_node", "subgraph_node")
parent_builder.add_edge("subgraph_node", END)

# 부모 그래프 컴파일
parent_graph = parent_builder.compile()

# 실행
result = parent_graph.invoke({"message": "안녕하세요"})
print(f"\n결과: {result['message']}")

---

# Part 2: Shared State Schemas 🔗

## 2.1 공유 상태 스키마의 개념

**Shared State Schemas**는 부모 그래프와 서브그래프가 **동일한 상태 키를 공유**하는 방식입니다. 이는 **멀티 에이전트 시스템**에서 에이전트들이 **공통 메시지 채널**을 통해 통신할 때 특히 유용합니다.

### 특징
- ✅ 부모와 자식이 같은 키에 읽기/쓰기 가능
- ✅ 서브그래프만의 프라이빗 키도 가질 수 있음
- ✅ 간단한 통신 구조

In [ ]:
from typing import List
from operator import add
from typing import Annotated

# 공유 상태 스키마 예제
class SharedState(TypedDict):
    """State shared between parent and subgraph"""
    shared_data: str  # 부모와 서브그래프가 공유하는 키
    messages: Annotated[List[str], add]  # 메시지 리스트 (누적)

class SubgraphPrivateState(TypedDict):
    """Subgraph state with private keys"""
    shared_data: str  # 부모와 공유
    messages: Annotated[List[str], add]  # 부모와 공유
    private_info: str  # 서브그래프만 접근 가능한 프라이빗 키

# 서브그래프 생성
def create_shared_subgraph():
    """Create subgraph with shared state"""
    builder = StateGraph(SubgraphPrivateState)
    
    def subgraph_node_1(state: SubgraphPrivateState):
        # 프라이빗 정보 설정
        return {
            "private_info": "이것은 서브그래프의 비밀 정보입니다",
            "messages": ["[서브그래프] 노드1 실행"]
        }
    
    def subgraph_node_2(state: SubgraphPrivateState):
        # 프라이빗 정보를 사용하여 공유 데이터 업데이트
        return {
            "shared_data": state["shared_data"] + f" + {state['private_info'][:10]}...",
            "messages": ["[서브그래프] 노드2 실행 - 프라이빗 정보 활용"]
        }
    
    # 노드와 엣지 추가
    builder.add_node("sub_node_1", subgraph_node_1)
    builder.add_node("sub_node_2", subgraph_node_2)
    builder.add_edge(START, "sub_node_1")
    builder.add_edge("sub_node_1", "sub_node_2")
    builder.add_edge("sub_node_2", END)
    
    return builder.compile()

# 부모 그래프 생성
parent_builder = StateGraph(SharedState)

def parent_start_node(state: SharedState):
    # 부모 그래프에서 초기 데이터 설정
    return {
        "shared_data": "부모 데이터",
        "messages": ["[부모] 시작 노드 실행"]
    }

def parent_end_node(state: SharedState):
    # 서브그래프 실행 후 최종 처리
    return {
        "messages": ["[부모] 종료 노드 실행"]
    }

# 노드 추가
parent_builder.add_node("start", parent_start_node)
parent_builder.add_node("subgraph", create_shared_subgraph())  # 서브그래프를 노드로
parent_builder.add_node("end", parent_end_node)

# 엣지 연결
parent_builder.add_edge(START, "start")
parent_builder.add_edge("start", "subgraph")
parent_builder.add_edge("subgraph", "end")
parent_builder.add_edge("end", END)

# 컴파일 및 실행
shared_graph = parent_builder.compile()

result = shared_graph.invoke({
    "shared_data": "초기값",
    "messages": []
})

print("\n🔗 Shared State Schema 실행 결과:")
print(f"\n최종 데이터: {result['shared_data']}")
print("\n실행 로그:")
for msg in result['messages']:
    print(f"  • {msg}")

---

# Part 3: Different State Schemas 🔄

## 3.1 독립적인 상태 스키마

**Different State Schemas**는 부모와 서브그래프가 **완전히 다른 상태 구조**를 가지는 방식입니다. 이 경우:

1. **상태 변환 필요**: 부모 → 서브그래프, 서브그래프 → 부모
2. **캡슐화 강화**: 서브그래프의 내부 구현을 완전히 숨김
3. **유연성 증가**: 각 그래프가 독립적으로 진화 가능

### 활용 예시
- 각 에이전트가 고유한 메시지 히스토리를 유지하는 멀티 에이전트 시스템
- 특정 도메인 로직을 캡슐화한 재사용 가능한 컴포넌트

In [ ]:
# 부모 그래프의 상태
class ParentState(TypedDict):
    """Parent graph state"""
    user_input: str
    final_output: str

# 서브그래프의 완전히 다른 상태
class SubgraphState(TypedDict):
    """Subgraph state - completely different from parent"""
    task_description: str
    processing_steps: List[str]
    result: str

# 서브그래프 생성
def create_different_subgraph():
    """Create subgraph with different state schema"""
    builder = StateGraph(SubgraphState)
    
    def analyze_task(state: SubgraphState):
        # 작업 분석
        return {
            "processing_steps": ["작업 분석 완료"],
            "result": f"분석: {state['task_description']}"
        }
    
    def process_task(state: SubgraphState):
        # 작업 처리
        return {
            "processing_steps": state["processing_steps"] + ["작업 처리 완료"],
            "result": state["result"] + " → 처리됨"
        }
    
    def finalize_task(state: SubgraphState):
        # 작업 완료
        return {
            "processing_steps": state["processing_steps"] + ["최종화 완료"],
            "result": f"✅ {state['result']} (단계: {len(state['processing_steps']) + 1})"
        }
    
    # 노드와 엣지 구성
    builder.add_node("analyze", analyze_task)
    builder.add_node("process", process_task)
    builder.add_node("finalize", finalize_task)
    
    builder.add_edge(START, "analyze")
    builder.add_edge("analyze", "process")
    builder.add_edge("process", "finalize")
    builder.add_edge("finalize", END)
    
    return builder.compile()

# 부모 그래프 생성
parent_builder = StateGraph(ParentState)

def prepare_input(state: ParentState):
    # 사용자 입력 준비
    return {"user_input": f"[준비됨] {state['user_input']}"}

def call_subgraph_with_transform(state: ParentState):
    """Call subgraph with state transformation"""
    # 1. 부모 상태 → 서브그래프 상태로 변환
    subgraph_input = {
        "task_description": state["user_input"],  # 변환 매핑
        "processing_steps": [],
        "result": ""
    }
    
    # 2. 서브그래프 실행
    subgraph = create_different_subgraph()
    subgraph_output = subgraph.invoke(subgraph_input)
    
    # 3. 서브그래프 결과 → 부모 상태로 변환
    return {
        "final_output": subgraph_output["result"],  # 역변환 매핑
    }

# 노드 추가
parent_builder.add_node("prepare", prepare_input)
parent_builder.add_node("subgraph_caller", call_subgraph_with_transform)

# 엣지 연결
parent_builder.add_edge(START, "prepare")
parent_builder.add_edge("prepare", "subgraph_caller")
parent_builder.add_edge("subgraph_caller", END)

# 컴파일 및 실행
different_graph = parent_builder.compile()

result = different_graph.invoke({
    "user_input": "복잡한 작업 요청",
    "final_output": ""
})

print("\n🔄 Different State Schema 실행 결과:")
print(f"\n입력: {result['user_input']}")
print(f"출력: {result['final_output']}")

## 3.2 다층 서브그래프 (Nested Subgraphs)

서브그래프 안에 또 다른 서브그래프를 포함시킬 수 있습니다. 이를 통해 **계층적 구조**의 복잡한 시스템을 구축할 수 있습니다.

In [ ]:
# 손자 그래프 상태
class GrandchildState(TypedDict):
    """Grandchild graph state"""
    detail_info: str

# 자식 그래프 상태
class ChildState(TypedDict):
    """Child graph state"""
    child_data: str

# 부모 그래프 상태
class NestedParentState(TypedDict):
    """Parent graph state"""
    main_data: str
    hierarchy_path: List[str]

# 손자 그래프 생성
def create_grandchild_graph():
    """Create grandchild graph"""
    builder = StateGraph(GrandchildState)
    
    def grandchild_process(state: GrandchildState):
        # 손자 레벨 처리
        return {"detail_info": state["detail_info"] + " [손자 처리]"}
    
    builder.add_node("gc_process", grandchild_process)
    builder.add_edge(START, "gc_process")
    builder.add_edge("gc_process", END)
    
    return builder.compile()

# 자식 그래프 생성 (손자 그래프 포함)
def create_child_graph():
    """Create child graph with grandchild"""
    builder = StateGraph(ChildState)
    
    def child_process(state: ChildState):
        # 자식 레벨 처리
        return {"child_data": state["child_data"] + " [자식 처리]"}
    
    def call_grandchild(state: ChildState):
        # 손자 그래프 호출 (상태 변환 포함)
        grandchild = create_grandchild_graph()
        gc_input = {"detail_info": state["child_data"]}
        gc_output = grandchild.invoke(gc_input)
        return {"child_data": gc_output["detail_info"]}
    
    builder.add_node("c_process", child_process)
    builder.add_node("call_gc", call_grandchild)
    
    builder.add_edge(START, "c_process")
    builder.add_edge("c_process", "call_gc")
    builder.add_edge("call_gc", END)
    
    return builder.compile()

# 부모 그래프 생성 (자식 그래프 포함)
nested_parent_builder = StateGraph(NestedParentState)

def parent_process(state: NestedParentState):
    # 부모 레벨 처리
    return {
        "main_data": state["main_data"] + " [부모 처리]",
        "hierarchy_path": ["부모"]
    }

def call_child_graph(state: NestedParentState):
    # 자식 그래프 호출 (상태 변환 포함)
    child = create_child_graph()
    child_input = {"child_data": state["main_data"]}
    child_output = child.invoke(child_input)
    
    return {
        "main_data": child_output["child_data"],
        "hierarchy_path": state["hierarchy_path"] + ["자식", "손자"]
    }

# 노드 추가
nested_parent_builder.add_node("p_process", parent_process)
nested_parent_builder.add_node("call_child", call_child_graph)

# 엣지 연결
nested_parent_builder.add_edge(START, "p_process")
nested_parent_builder.add_edge("p_process", "call_child")
nested_parent_builder.add_edge("call_child", END)

# 컴파일 및 실행
nested_graph = nested_parent_builder.compile()

result = nested_graph.invoke({
    "main_data": "초기 데이터",
    "hierarchy_path": []
})

print("\n🏗️ 다층 서브그래프 실행 결과:")
print(f"\n최종 데이터: {result['main_data']}")
print(f"계층 경로: {' → '.join(result['hierarchy_path'])}")

---

# Part 4: Persistence와 메모리 관리 💾

## 4.1 체크포인터 전파

서브그래프의 메모리 관리는 매우 중요합니다. LangGraph는 **부모 그래프의 체크포인터를 자동으로 서브그래프에 전파**합니다.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

class PersistentState(TypedDict):
    """State for persistence example"""
    counter: int
    history: List[str]

# 서브그래프 생성
def create_persistent_subgraph():
    """Create subgraph with persistence capability"""
    builder = StateGraph(PersistentState)
    
    def increment_counter(state: PersistentState):
        # 카운터 증가
        return {
            "counter": state["counter"] + 1,
            "history": state["history"] + [f"서브그래프: 카운터 {state['counter']} → {state['counter'] + 1}"]
        }
    
    builder.add_node("increment", increment_counter)
    builder.add_edge(START, "increment")
    builder.add_edge("increment", END)
    
    # 서브그래프에 독립적인 메모리를 원한다면:
    # return builder.compile(checkpointer=True)
    
    # 부모의 체크포인터를 공유하려면:
    return builder.compile()

# 부모 그래프 생성
persistent_parent_builder = StateGraph(PersistentState)

def parent_increment(state: PersistentState):
    # 부모에서도 카운터 증가
    return {
        "counter": state["counter"] + 10,
        "history": state["history"] + [f"부모: 카운터 +10"]
    }

# 노드 추가
persistent_parent_builder.add_node("parent_inc", parent_increment)
persistent_parent_builder.add_node("subgraph", create_persistent_subgraph())

# 엣지 연결
persistent_parent_builder.add_edge(START, "parent_inc")
persistent_parent_builder.add_edge("parent_inc", "subgraph")
persistent_parent_builder.add_edge("subgraph", END)

# 체크포인터와 함께 컴파일
memory = MemorySaver()
persistent_graph = persistent_parent_builder.compile(checkpointer=memory)

# 첫 번째 실행
config = {"configurable": {"thread_id": "test-thread-1"}}
result1 = persistent_graph.invoke(
    {"counter": 0, "history": []},
    config=config
)

print("\n💾 Persistence 테스트:")
print("\n첫 번째 실행:")
print(f"  카운터: {result1['counter']}")
print(f"  히스토리: {result1['history']}")

# 두 번째 실행 (같은 thread_id로 상태 복원)
result2 = persistent_graph.invoke(
    None,  # None을 전달하면 이전 상태에서 계속
    config=config
)

print("\n두 번째 실행 (상태 복원):")
print(f"  카운터: {result2['counter']}")
print(f"  히스토리 길이: {len(result2['history'])}개")

---

# Part 5: Subgraph State 모니터링 🔍

## 5.1 중단된 서브그래프 상태 보기

서브그래프가 **중단(interrupt)된 상태**일 때, 그 내부 상태를 조회할 수 있습니다. 이는 디버깅과 모니터링에 매우 유용합니다.

⚠️ **중요**: 서브그래프 상태는 **오직 중단된 상태에서만** 조회 가능합니다!

In [ ]:
from langgraph.types import interrupt, Command
from typing import Literal

class InterruptState(TypedDict):
    """State for interrupt example"""
    value: str
    approved: bool

# 중단이 있는 서브그래프
def create_interruptible_subgraph():
    """Create subgraph with interrupt"""
    builder = StateGraph(InterruptState)
    
    def request_approval(state: InterruptState):
        # 사용자 승인 요청 (중단)
        user_input = interrupt("이 작업을 승인하시겠습니까? (yes/no)")
        return {
            "value": state["value"] + f" - 사용자 응답: {user_input}",
            "approved": user_input.lower() == "yes"
        }
    
    builder.add_node("request", request_approval)
    builder.add_edge(START, "request")
    builder.add_edge("request", END)
    
    return builder.compile()

# 부모 그래프
interrupt_parent_builder = StateGraph(InterruptState)

def prepare_request(state: InterruptState):
    # 요청 준비
    return {"value": "중요한 작업"}

# 노드 추가
interrupt_parent_builder.add_node("prepare", prepare_request)
interrupt_parent_builder.add_node("subgraph", create_interruptible_subgraph())

# 엣지 연결
interrupt_parent_builder.add_edge(START, "prepare")
interrupt_parent_builder.add_edge("prepare", "subgraph")
interrupt_parent_builder.add_edge("subgraph", END)

# 체크포인터와 함께 컴파일
memory = MemorySaver()
interrupt_graph = interrupt_parent_builder.compile(checkpointer=memory)

# 실행 (중단될 것임)
config = {"configurable": {"thread_id": "interrupt-test"}}

print("\n🔍 Interrupt 모니터링 테스트:")
print("\n1. 첫 실행 (중단됨):")

try:
    result = interrupt_graph.invoke(
        {"value": "", "approved": False},
        config=config
    )
except Exception as e:
    print(f"  중단됨: {e}")

# 상태 조회
parent_state = interrupt_graph.get_state(config)
print(f"\n2. 부모 그래프 상태:")
print(f"  다음 실행할 노드: {parent_state.next}")

# 서브그래프 상태 조회 (중단된 상태에서만 가능)
full_state = interrupt_graph.get_state(config, subgraphs=True)
if full_state.tasks:
    subgraph_state = full_state.tasks[0].state
    print(f"\n3. 서브그래프 상태:")
    print(f"  서브그래프 다음 노드: {subgraph_state.next if subgraph_state else 'N/A'}")

# 재개 (사용자 응답 제공)
print("\n4. 재개 (승인):")
result = interrupt_graph.invoke(
    Command(resume="yes"),
    config=config
)

print(f"  최종 결과: {result['value']}")
print(f"  승인 여부: {result['approved']}")

---

# Part 6: Stream Outputs 📡

## 6.1 서브그래프 출력 스트리밍

서브그래프의 실행 과정을 실시간으로 모니터링하려면 `subgraphs=True` 옵션을 사용합니다.

In [ ]:
class StreamState(TypedDict):
    """State for streaming example"""
    data: str
    steps: List[str]

# 여러 단계가 있는 서브그래프
def create_multi_step_subgraph():
    """Create subgraph with multiple steps"""
    builder = StateGraph(StreamState)
    
    def step1(state: StreamState):
        # 단계 1
        return {
            "data": state["data"] + " → 단계1",
            "steps": state["steps"] + ["서브그래프 단계1 완료"]
        }
    
    def step2(state: StreamState):
        # 단계 2
        return {
            "data": state["data"] + " → 단계2",
            "steps": state["steps"] + ["서브그래프 단계2 완료"]
        }
    
    def step3(state: StreamState):
        # 단계 3
        return {
            "data": state["data"] + " → 단계3",
            "steps": state["steps"] + ["서브그래프 단계3 완료"]
        }
    
    # 노드 추가
    builder.add_node("step1", step1)
    builder.add_node("step2", step2)
    builder.add_node("step3", step3)
    
    # 순차 실행
    builder.add_edge(START, "step1")
    builder.add_edge("step1", "step2")
    builder.add_edge("step2", "step3")
    builder.add_edge("step3", END)
    
    return builder.compile()

# 부모 그래프
stream_parent_builder = StateGraph(StreamState)

def parent_init(state: StreamState):
    # 부모 초기화
    return {
        "data": "시작",
        "steps": ["부모 초기화 완료"]
    }

def parent_finalize(state: StreamState):
    # 부모 최종화
    return {
        "data": state["data"] + " → 완료",
        "steps": state["steps"] + ["부모 최종화 완료"]
    }

# 노드 추가
stream_parent_builder.add_node("init", parent_init)
stream_parent_builder.add_node("subgraph", create_multi_step_subgraph())
stream_parent_builder.add_node("finalize", parent_finalize)

# 엣지 연결
stream_parent_builder.add_edge(START, "init")
stream_parent_builder.add_edge("init", "subgraph")
stream_parent_builder.add_edge("subgraph", "finalize")
stream_parent_builder.add_edge("finalize", END)

# 컴파일
stream_graph = stream_parent_builder.compile()

print("\n📡 스트리밍 출력 테스트:")
print("\n1. 일반 스트리밍 (부모만):")

for chunk in stream_graph.stream(
    {"data": "", "steps": []},
    stream_mode="updates"
):
    print(f"  {chunk}")

print("\n2. 서브그래프 포함 스트리밍:")

for chunk in stream_graph.stream(
    {"data": "", "steps": []},
    stream_mode="updates",
    subgraphs=True  # 서브그래프 출력 포함!
):
    namespace, update = chunk
    if namespace:
        print(f"  [서브그래프] {update}")
    else:
        print(f"  [부모] {update}")

---

# Part 7: 실전 - 멀티 에이전트 시스템 🤖

## 7.1 전문가 에이전트 시스템

서브그래프를 활용하여 각 분야의 전문가 에이전트로 구성된 멀티 에이전트 시스템을 구축해봅시다.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langgraph.graph.message import add_messages

# 공유 메시지 상태
class AgentState(TypedDict):
    """Shared state for multi-agent system"""
    messages: Annotated[List[AnyMessage], add_messages]
    current_agent: str
    task_type: str
    metadata: dict

# 연구원 에이전트 (서브그래프)
def create_researcher_agent():
    """Create researcher agent subgraph"""
    builder = StateGraph(AgentState)
    
    def research(state: AgentState):
        # 연구 수행
        last_message = state["messages"][-1]
        research_result = f"📚 연구 결과: '{last_message.content}'에 대한 심층 분석 완료"
        
        return {
            "messages": [AIMessage(content=research_result)],
            "metadata": {**state.get("metadata", {}), "research_done": True}
        }
    
    def validate_research(state: AgentState):
        # 연구 검증
        validation = "✅ 연구 검증: 신뢰도 95%, 출처 확인 완료"
        return {
            "messages": [AIMessage(content=validation)],
            "metadata": {**state.get("metadata", {}), "validated": True}
        }
    
    builder.add_node("research", research)
    builder.add_node("validate", validate_research)
    
    builder.add_edge(START, "research")
    builder.add_edge("research", "validate")
    builder.add_edge("validate", END)
    
    return builder.compile()

# 개발자 에이전트 (서브그래프)
def create_developer_agent():
    """Create developer agent subgraph"""
    builder = StateGraph(AgentState)
    
    def analyze_requirements(state: AgentState):
        # 요구사항 분석
        last_message = state["messages"][-1]
        analysis = f"💻 요구사항 분석: '{last_message.content}'를 위한 기술 스택 결정"
        
        return {
            "messages": [AIMessage(content=analysis)],
            "metadata": {**state.get("metadata", {}), "tech_stack": "Python, LangGraph"}
        }
    
    def generate_code(state: AgentState):
        # 코드 생성
        code = """```python
# 자동 생성된 코드
def solution():
    return "구현 완료"
```"""
        
        return {
            "messages": [AIMessage(content=f"코드 생성 완료:\n{code}")],
            "metadata": {**state.get("metadata", {}), "code_generated": True}
        }
    
    builder.add_node("analyze", analyze_requirements)
    builder.add_node("code", generate_code)
    
    builder.add_edge(START, "analyze")
    builder.add_edge("analyze", "code")
    builder.add_edge("code", END)
    
    return builder.compile()

# 매니저 에이전트 (서브그래프)
def create_manager_agent():
    """Create manager agent subgraph"""
    builder = StateGraph(AgentState)
    
    def review_work(state: AgentState):
        # 작업 검토
        messages = state["messages"]
        review = "🎯 매니저 검토: 모든 작업이 성공적으로 완료되었습니다."
        
        # 작업 요약
        summary = "\n📋 작업 요약:\n"
        for msg in messages[-3:]:
            if isinstance(msg, AIMessage):
                summary += f"  • {msg.content[:50]}...\n"
        
        return {
            "messages": [AIMessage(content=review + summary)],
            "metadata": {**state.get("metadata", {}), "reviewed": True}
        }
    
    builder.add_node("review", review_work)
    builder.add_edge(START, "review")
    builder.add_edge("review", END)
    
    return builder.compile()

# 메인 조정자 그래프
coordinator_builder = StateGraph(AgentState)

def route_task(state: AgentState) -> Literal["researcher", "developer", "manager"]:
    """Route task to appropriate agent"""
    # 작업 유형에 따라 적절한 에이전트로 라우팅
    task_type = state.get("task_type", "")
    
    if "연구" in task_type or "조사" in task_type:
        return "researcher"
    elif "개발" in task_type or "코드" in task_type:
        return "developer"
    else:
        return "manager"

def classify_task(state: AgentState):
    """Classify the incoming task"""
    # 작업 분류
    last_message = state["messages"][-1]
    content = last_message.content.lower()
    
    if "연구" in content or "조사" in content:
        task_type = "연구"
    elif "개발" in content or "코드" in content or "구현" in content:
        task_type = "개발"
    else:
        task_type = "관리"
    
    return {
        "task_type": task_type,
        "messages": [AIMessage(content=f"📌 작업 분류: {task_type} 작업으로 분류됨")]
    }

# 노드 추가
coordinator_builder.add_node("classify", classify_task)
coordinator_builder.add_node("researcher", create_researcher_agent())
coordinator_builder.add_node("developer", create_developer_agent())
coordinator_builder.add_node("manager", create_manager_agent())

# 엣지 추가
coordinator_builder.add_edge(START, "classify")
coordinator_builder.add_conditional_edges(
    "classify",
    route_task,
    {
        "researcher": "researcher",
        "developer": "developer",
        "manager": "manager"
    }
)

# 모든 에이전트는 매니저로 최종 검토
coordinator_builder.add_edge("researcher", "manager")
coordinator_builder.add_edge("developer", "manager")
coordinator_builder.add_edge("manager", END)

# 컴파일
multi_agent_system = coordinator_builder.compile()

print("\n🤖 멀티 에이전트 시스템 테스트:\n")

# 다양한 작업 테스트
test_tasks = [
    "LangGraph에 대해 연구해주세요",
    "간단한 챗봇 코드를 개발해주세요",
    "프로젝트 진행 상황을 검토해주세요"
]

for task in test_tasks:
    print(f"\n📝 작업: {task}")
    print("=" * 50)
    
    result = multi_agent_system.invoke({
        "messages": [HumanMessage(content=task)],
        "current_agent": "coordinator",
        "task_type": "",
        "metadata": {}
    })
    
    # 최종 메시지 출력
    for msg in result["messages"][-3:]:
        if isinstance(msg, AIMessage):
            print(f"\n{msg.content}")
    
    print(f"\n📊 메타데이터: {result['metadata']}")

---

# 🎉 축하합니다!

LangGraph Subgraph의 핵심 개념을 모두 학습했습니다!

## ✅ 학습 완료 항목

1. **Subgraph 기본 개념** - 그래프 안의 그래프 구조
2. **Shared State Schemas** - 부모와 자식 간 상태 공유
3. **Different State Schemas** - 독립적인 상태와 변환
4. **다층 서브그래프** - 중첩된 그래프 구조
5. **Persistence** - 메모리 관리와 체크포인터
6. **모니터링** - 중단된 서브그래프 상태 조회
7. **스트리밍** - 실시간 출력 추적
8. **멀티 에이전트 시스템** - 실전 활용 예제

## 🚀 다음 단계

### 실전 적용 아이디어

1. **모듈화된 워크플로우**
   - 재사용 가능한 비즈니스 로직 컴포넌트
   - 플러그인 시스템 구축

2. **확장 가능한 멀티 에이전트**
   - 동적으로 에이전트 추가/제거
   - 에이전트 간 복잡한 협업 패턴

3. **계층적 의사결정 시스템**
   - 상위 레벨 전략과 하위 레벨 실행 분리
   - 복잡한 승인 워크플로우

### 추가 학습 자료

- **동적 서브그래프**: 런타임에 서브그래프 생성
- **서브그래프 풀**: 서브그래프 인스턴스 관리
- **분산 서브그래프**: 네트워크를 통한 서브그래프 실행
- **성능 최적화**: 서브그래프 캐싱과 병렬 처리

## 💡 핵심 팁

1. **상태 설계 우선**: Shared vs Different 스키마 선택이 중요
2. **캡슐화 활용**: 복잡한 로직은 서브그래프로 격리
3. **재사용성 고려**: 범용적인 서브그래프 설계
4. **모니터링 구축**: 스트리밍과 상태 조회 활용
5. **점진적 복잡도**: 단순한 서브그래프부터 시작

## 🎯 실습 과제

다음 시나리오를 서브그래프로 구현해보세요:

1. **고객 지원 시스템**: 문의 분류 → 전문가 배정 → 해결책 제공
2. **데이터 파이프라인**: 수집 → 변환 → 검증 → 저장
3. **승인 워크플로우**: 요청 → 검토 → 승인/거부 → 알림

이제 여러분은 LangGraph Subgraph를 활용하여 복잡한 멀티 에이전트 시스템과 모듈화된 워크플로우를 구축할 준비가 되었습니다! 🚀